In [2]:

import mysql.connector as connector
connection=connector.connect(
                             host="localhost",
                             user="root",
                             password="123456",
                            )
cursor = connection.cursor()
cursor.execute("USE little_lemon")

In [3]:

#STORED PROCEDURE: PeakHours
cursor.execute("DROP PROCEDURE IF EXISTS PeakHours;")
peakhours_query="""
CREATE PROCEDURE PeakHours()
BEGIN
    SELECT 
    HOUR(BookingSlot) AS booking_hour,
    COUNT(HOUR(BookingSlot)) AS n_bookings
    FROM Bookings
    GROUP BY booking_hour
    ORDER BY n_bookings DESC;
END"""
cursor.execute(peakhours_query)

In [4]:
cursor.callproc("PeakHours")
results = next(cursor.stored_results() )
dataset = results.fetchall()
#retrieve column
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

print(columns)
for data in dataset:
    print(data)

['booking_hour', 'n_bookings']
(19, 4)
(18, 2)
(15, 1)
(17, 1)
(20, 1)


In [5]:
#STORED PROCEDURE: GuestStatus
cursor.execute("DROP PROCEDURE IF EXISTS GuestStatus;")
gueststatus_query="""
CREATE PROCEDURE GuestStatus()
BEGIN
SELECT 
    Bookings.BookingID AS OrderNumber,  
    CONCAT(GuestFirstName,' ',GuestLastName) AS GuestName, 
    Role AS Employee, 
    CASE 
        WHEN Role IN ('Manager','Assistant Manager') THEN "Ready to Pay"
        WHEN Role = 'Head Chef' THEN "Ready to serve"
        WHEN Role = 'Assistant Chef' THEN "Preparing order"
        WHEN Role = 'Head Waiter' THEN "Order served"
    ELSE "Pending"
    END AS Status
    FROM Bookings 
        LEFT JOIN 
        Employees 
        ON Employees.EmployeeID=Bookings.EmployeeID;
END"""
cursor.execute(gueststatus_query)

In [6]:
cursor.callproc("GuestStatus")
results = next(cursor.stored_results())
dataset = results.fetchall()

# Retrieve column 
for column_id in cursor.stored_results():
    columns = [column[0] for column in column_id.description]

print(columns)
for data in dataset:
    print(data)
    


['OrderNumber', 'GuestName', 'Employee', 'Status']
(1, 'Anna Iversen', 'Manager', 'Ready to Pay')
(2, 'Joakim Iversen', 'Manager', 'Ready to Pay')
(3, 'Vanessa McCarthy', 'Head Chef', 'Ready to serve')
(4, 'Marcos Romero', 'Assistant Chef', 'Preparing order')
(5, 'Hiroki Yamane', 'Assistant Manager', 'Ready to Pay')
(6, 'Diana Pinto', 'Head Waiter', 'Order served')
(7, 'Anees Java', 'Receptionist', 'Pending')
(8, 'Bald Vin', 'Receptionist', 'Pending')
(9, 'Jay Kon', 'Receptionist', 'Pending')


In [7]:


#Little Lemon Analysis and Sales Report

import mysql.connector as connector

from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

# Define database configurations
dbconfig = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "little_lemon",
}

# Establish a connection pool with two connections
pool= MySQLConnectionPool(pool_name="mypool", pool_size=2, **dbconfig)

# Implement error handling
try:
    # Get a connection from the pool
    connection = pool.get_connection()

    # Perform database operations here
    # ...
    print("The connection pool is created with a name: ",pool.pool_name)
    print("The pool size is:",pool.pool_size)
    # Release the connection back to the pool
    connection.close()
except Error as e:
    print("Error while connecting to MySQL:", e)


The connection pool is created with a name:  mypool
The pool size is: 2


In [8]:
# Define the data for the three guests
guests = [
    {
        "table_number": 8,
        "first_name": "Anees",
        "last_name": "Java",
        "booking_time": "18:00",
        "employee_id": 6
    },
    {
        "table_number": 5,
        "first_name": "Bald",
        "last_name": "Vin",
        "booking_time": "19:00",
        "employee_id": 6
    },
    {
        "table_number": 12,
        "first_name": "Jay",
        "last_name": "Kon",
        "booking_time": "19:30",
        "employee_id": 6
    }
]

# Insert the data for each guest in the Bookings table
for guest in guests:
    try:
        # Get a connection from the pool
        connection = pool.get_connection()

        # Execute the INSERT query
        cursor = connection.cursor()
        query = "INSERT INTO Bookings (TableNo, FirstName, LastName, BookingTime, EmployeeID) VALUES (%s, %s, %s, %s, %s)"
        data = (guest["table_number"], guest["first_name"], guest["last_name"], guest["booking_time"], guest["employee_id"])
        cursor.execute(query, data)

        # Commit the changes
        connection.commit()

        # Release the connection back to the pool
        connection.close()
    except Error as e:
        print("Error while connecting to MySQL:", e)

Error while connecting to MySQL: 1054 (42S22): Unknown column 'FirstName' in 'field list'
Error while connecting to MySQL: 1054 (42S22): Unknown column 'FirstName' in 'field list'
Error while connecting to MySQL: Failed getting connection; pool exhausted


In [9]:
#Create report
select_name_id_query = """
SELECT Name, EmployeeID 
FROM Employees WHERE Role = 'Manager'
"""
cursor.execute(select_name_id_query)
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

('Name', 'EmployeeID')
('Mario Gollini', 1)


In [10]:
select_name_role_query = """
SELECT Name, Role 
FROM Employees WhERE Annual_Salary = (SELECT MAX(Annual_Salary) FROM Employees)
"""
cursor.execute(select_name_role_query)
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

('Name', 'Role')
('Mario Gollini', 'Manager')


In [11]:
select_number_guest_booked_query = """
SELECT COUNT(BookingID)
FROM bookings 
WHERE BookingSlot between '18:00:00' and '20:00:00'
"""
cursor.execute(select_number_guest_booked_query)
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

('COUNT(BookingID)',)
(7,)


In [12]:
select_guest_waiting_query="""
SELECT 
    b.BookingID AS ID,  
    CONCAT(GuestFirstName,' ',GuestLastName) AS GuestName, 
    Role AS Employee
FROM Bookings b
    LEFT JOIN Employees e
    ON e.EmployeeID=b.EmployeeID
    WHERE e.Role = "Receptionist"
    ORDER BY BookingSlot DESC;
"""
cursor.execute(select_guest_waiting_query)
results=cursor.fetchall()
columns=cursor.column_names
print(columns)
for result in results:
    print(result)

('ID', 'GuestName', 'Employee')
(9, 'Jay Kon', 'Receptionist')
(8, 'Bald Vin', 'Receptionist')
(7, 'Anees Java', 'Receptionist')


In [13]:
#Stored Procedure Basic Sales Report 

cursor.execute("DROP PROCEDURE IF EXISTS BasicSalesReport;")

stored_procedure_query="""
CREATE PROCEDURE BasicSalesReport()
BEGIN
SELECT 
    SUM(BillAmount) AS Total_Sales,
    AVG(BillAmount) AS Average_Sale,
    MIN(BillAmount) AS Minimum_Bill_Paid,
    MAX(BillAmount) AS Maximum_Bill_Paid
FROM Orders;
END"""
cursor.execute(stored_procedure_query)
cursor.callproc("BasicSalesReport")

# Retrieve records in "dataset"
results = next( cursor.stored_results() )
dataset = results.fetchall()

print("Sales Report")
for data in dataset:
    print(data)

Sales Report
(Decimal('243'), Decimal('48.6000'), 37, 86)


In [14]:
#Upcoming Bookings
import mysql.connector as connector

from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

# Define database configurations
dbconfig = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "little_lemon",
}

# Establish a connection pool with two connections
pool= MySQLConnectionPool(pool_name="mypool", pool_size=2, **dbconfig)

connection1 = pool.get_connection()
cursor = connection1.cursor()

select_upcoming_bookings_query = """
SELECT 
	CONCAT("BookingSlot"," ", b.BookingSlot) as BookingSlot,
	CONCAT("Guest_name:"," ", CONCAT(b.GuestFirstName," ",b.GuestLastName)) as Guest_name,
	CONCAT("Assigned_to:"," ", CONCAT(e.Name, " [", e.Role, "]")) AS Assigned_to
FROM Bookings b
INNER JOIN 
Employees e
ON e.EmployeeID=b.EmployeeID
Order by b.BookingID desc limit 3;
"""

cursor.execute(select_upcoming_bookings_query)
results = cursor.fetchall()
for result in results:
    print(result)


('BookingSlot 19:30:00', 'Guest_name: Jay Kon', 'Assigned_to: John Millar [Receptionist]')
('BookingSlot 19:00:00', 'Guest_name: Bald Vin', 'Assigned_to: John Millar [Receptionist]')
('BookingSlot 18:00:00', 'Guest_name: Anees Java', 'Assigned_to: John Millar [Receptionist]')


In [15]:
cursor.execute("""SELECT ADDTIME("1:00:00","-1:00:00")""")
result=cursor.fetchall()
print(result)